(function:open-converted)=
# Working with converted files

## Open a converted netCDF or Zarr dataset

Converted netCDF files can be opened with the `open_converted` function that returns a lazy-loaded [`EchoData` object](data-format:echodata-object) (only metadata are read during opening):

```python
import echopype as ep
file_path = "./converted_files/file.nc"      # path to a converted nc file
ed = ep.open_converted(file_path)            # create an EchoData object
```

Likewise, specify the path to open a Zarr dataset. To open such a dataset from cloud storage, use the same `storage_options` parameter as with [open_raw](convert.html#aws-s3-access). For example:

```python
s3_path = "s3://s3bucketname/directory_path/dataset.zarr"     # S3 dataset path
ed = ep.open_converted(s3_path, storage_options={"anon": True})
```

## Combine EchoData objects

Converted data found in multiple files corresponding to the same instrument deployment can be combined into a single `EchoData` object. First assemble a list of [`EchoData` objects](data-format:echodata-object) from the converted files (netCDF or Zarr). Then apply `combine_echodata` on this list to combine all the data into a single `EchoData` object in memory:

```python
ed_list = []
for converted_file in ["convertedfile1.nc", "convertedfile2.nc"]:
   ed_list.append(ep.open_converted(converted_file))

combined_ed = ep.combine_echodata(ed_list)
```

:::{Warning}
We are aware that the `combine_echodata` method may result in memory problems when the [`EchoData` objects](data-format:echodata-object) hold large amount of data. This is an issue we are actively working on.
:::